In [1]:
from datasets import load_from_disk, load_metric
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig, Trainer
import numpy as np
import pyarrow as pa
import pandas as pd

#import os
#import sys
#sys.path.insert(0, '/zhome/a6/6/127219/Speciale/master_project')
#from src.models.transformers_modeling_roberta import RobertaForSequenceClassification

In [2]:
# load
datadir = '/work3/s174498/sst2_dataset/'
checkpoint = "/work3/s174498/test/linear_head_1/checkpoint-1000"

# test data
test_dataset = load_from_disk(datadir + 'test_dataset')

# tokenizer
tokenizer = RobertaTokenizer.from_pretrained(checkpoint)
tokenizer.model_max_len=512

# model
#config = RobertaConfig.from_pretrained(checkpoint, output_attentions=True, output_hidden_states=True)
config = RobertaConfig.from_pretrained(checkpoint)
config.output_hidden_states = True
#config.output_attentions= True
model = RobertaForSequenceClassification.from_pretrained(checkpoint, config=config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /work3/s174498/test/linear_head_1/checkpoint-1000 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, max_length=512)

In [4]:
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /work3/s174498/sst2_dataset/test_dataset/cache-e6d548d78ecc5fee.arrow


In [8]:
tokenized_test['input_ids']

[[0,
  102,
  1099,
  1203,
  77,
  51,
  128,
  241,
  3518,
  7,
  28,
  519,
  10,
  6981,
  1144,
  908,
  1437,
  2],
 [0,
  8585,
  16,
  10,
  1402,
  1472,
  9,
  33762,
  8,
  36403,
  1258,
  7,
  42,
  822,
  14,
  189,
  45,
  460,
  173,
  2156,
  53,
  24,
  16,
  21507,
  9669,
  479,
  1437,
  2],
 [0,
  1090,
  991,
  540,
  9,
  10,
  2664,
  30715,
  114,
  43601,
  4,
  4696,
  397,
  1432,
  149,
  15,
  69,
  25442,
  9,
  5,
  22636,
  24262,
  833,
  1437,
  2],
 [0,
  12963,
  114,
  42327,
  16,
  45,
  1341,
  5,
  756,
  4996,
  14,
  5,
  23674,
  661,
  16,
  13,
  910,
  16187,
  8385,
  24345,
  2156,
  24,
  13905,
  14,
  885,
  254,
  1396,
  69,
  329,
  2154,
  64,
  202,
  989,
  201,
  19,
  10,
  1472,
  9,
  5170,
  23,
  5,
  5544,
  2156,
  25591,
  9412,
  22027,
  16499,
  750,
  34,
  551,
  479,
  1437,
  2],
 [0,
  3865,
  6342,
  18711,
  444,
  350,
  203,
  15,
  5,
  11789,
  3222,
  5785,
  8,
  18672,
  1762,
  9,
  11877,
  227,
  

In [19]:
trainer = Trainer(
    model=model,                        
    tokenizer=tokenizer
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [20]:
# Predicting with model
predictions = trainer.predict(tokenized_test)


The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1821
  Batch size = 8


In [21]:
pred_dataset_test = list(np.argmax(predictions.predictions[0], axis=-1))
true_dataset_test = predictions.label_ids

In [23]:
# compute accuracy
accuracy_metric = load_metric("accuracy")
accuracy_metric.compute(predictions=pred_dataset_test, references=true_dataset_test)

{'accuracy': 0.4914881933003844}